In [1]:
from backtesting import Backtest, Strategy

/Users/jerald/opt/miniconda3/envs/btester/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/jerald/opt/miniconda3/envs/btester/lib/python3.11/site-packages/backtesting/_plotting.py:55: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support, such as old IDEs. Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [2]:
import pandas as pd 
import numpy as np 
from stockData import StockData
from bin.min import get_path 
from bin.min import Manager
from bin.utils.tools import pretty_print

connections = get_path()
man = Manager(connections)

def get_stock_data(manager, stock, start_date = "2025-01-01"):
    sd = StockData(manager = manager, stock = stock)
    df = sd.price_data[start_date:]
    df.columns = [x.capitalize() for x in df.columns]
    return df 

2025-05-06 16:52:56,281 - bin.options.optgd.db_connect - INFO - Options DB Initialized
2025-05-06 16:52:56,282 - bin.options.optgd.db_connect - WARNING - You currently have 8 connections open.
2025-05-06 16:52:56,284 - INFO - PriceDB Initialized successfully at 2025-05-06 16:52:56.284211
2025-05-06 16:52:56,284 - bin.price.db_connect - INFO - PriceDB Initialized successfully at 2025-05-06 16:52:56.284211
2025-05-06 16:52:56,284 - INFO - Established 3 database connections
2025-05-06 16:52:56,284 - bin.price.db_connect - INFO - Established 3 database connections


In [3]:
class OracleStrategy(Strategy):
    def init(self):
        """
        Precompute buy/sell signals based on future price knowledge.
        Signal: 1 (buy) if next close > current close, -1 (sell/short) if next close < current close.
        """
        close = self.data.Close
        future_close = pd.Series(close).shift(-1)  # Next bar's closing price
        # Create signals: 1 for buy, -1 for sell/short, 0 for hold (if prices are equal)
        self.signals = np.where(close < future_close, 1, np.where(close > future_close, -1, 0))
        self.signals = pd.Series(self.signals, index=self.data.index)

    def next(self):
        """
        Execute trades based on the precomputed signal for the current bar.
        Buy for +1, sell/short for -1, do nothing for 0.
        """
        current_signal = self.signals[self.data.index[-1]]
        if current_signal == 1:  # Next price is higher -> buy
            self.position.close()  # Close any existing position
            self.buy()
        elif current_signal == -1:  # Next price is lower -> sell/short
            self.position.close()  # Close any existing position
            self.sell()

In [4]:

df = get_stock_data(manager=man, stock = 'iwm', start_date = "2024-04-22")  # Use your manager-based call if needed: get_stock_data(man, 'UBER')
# Run the backtest
bt = Backtest(
    df,
    OracleStrategy,
    cash=10000,              # Initial cash
    commission=0.00,        # 0.2% commission per trade
    exclusive_orders=True,   # Only one position at a time
    trade_on_close=True     # Trade on next bar's open (standard for max profit)
)
stats = bt.run()

# Print key results
print("Backtest Results:")
print(f"Total Return: {stats['Return [%]']:,.2f}%")
print(f"Final Equity: ${stats['Equity Final [$]']:,.2f}")
print(f"Max Drawdown: {stats['Max. Drawdown [%]']:,.2f}%")
print(f"Sharpe Ratio: {stats['Sharpe Ratio']:.2f}")
print(f"Number of Trades: {stats['# Trades']:,.0f}")
print(f"Win Rate: {stats['Win Rate [%]']:.2f}%")
print(f"Duration: {stats['Duration']}")
print(f"Buy & Hold Return: {stats['Buy & Hold Return [%]']:,.2f}%")

Backtest Results:
Total Return: 1,422.09%
Final Equity: $152,209.36
Max Drawdown: -0.00%
Sharpe Ratio: 5.58
Number of Trades: 257
Win Rate: 100.00%
Duration: 378 days 00:00:00
Buy & Hold Return: 3.55%


In [5]:
stats

Start                     2024-04-22 00:00:00
End                       2025-05-05 00:00:00
Duration                    378 days 00:00:00
Exposure Time [%]                    99.23077
Equity Final [$]                 152209.35745
Equity Peak [$]                  152209.35745
Return [%]                         1422.09357
Buy & Hold Return [%]                  3.5548
Return (Ann.) [%]                    1299.776
Volatility (Ann.) [%]               232.91891
CAGR [%]                            514.17797
Sharpe Ratio                          5.58038
Sortino Ratio                             inf
Calmar Ratio                              NaN
Alpha [%]                          1422.02229
Beta                                  0.02005
Max. Drawdown [%]                        -0.0
Avg. Drawdown [%]                         NaN
Max. Drawdown Duration                    NaN
Avg. Drawdown Duration                    NaN
# Trades                                  257
Win Rate [%]                      